In [1]:
#load "Paket.fsx"
Paket.Package
  [ "QBitNinja.Client -Version 1.0.3.39" 
    "NBitcoin -Version 4.0.0.48"
  ]   
#load "Paket.Generated.Refs.fsx"


open NBitcoin 
open QBitNinja.Client 
open QBitNinja.Client.Models 
open System 
open System.Collections.Generic 


In [3]:

let client:QBitNinjaClient  = new QBitNinjaClient(Network.TestNet)

// Parse transaction id to NBitcoin.uint256 so the client can eat it

let transactionId = uint256.Parse("516c43b617a6fa6dee5f3e9b34d99365b3b89d1381dc9aebcca1e8822c8cb5f4")

// Query the transaction
let transactionResponse:GetTransactionResponse  = client.GetTransaction(transactionId).Result

let transaction:NBitcoin.Transaction  = transactionResponse.Transaction

let outputsCoins:TxOutList = transaction.Outputs 

let mutable myBitcoinPubKeys = new List<BitcoinAddress>()

for out:TxOut  in outputsCoins do
       let amount  = out.Value
       printfn "amount: %A" (amount.ToDecimal(MoneyUnit.BTC))
     
       let paymentScript = out.ScriptPubKey 
       printfn "pub script: %A" paymentScript // It's the ScriptPubKey
       let address = paymentScript.GetDestinationAddress(Network.TestNet)
       myBitcoinPubKeys.Add(address)
       printfn "destination address: %A" address 


amount: 0.55M
pub script: OP_DUP OP_HASH160 4fbd5152456bee3dc0ffe25118a6de17126e5c19 OP_EQUALVERIFY OP_CHECKSIG
destination address: mnnaUEKBKJVFjCqVnVPmwefk6CwuVsyRRj
amount: 30.30840894M
pub script: OP_HASH160 bbdaa6cdc5793ab520c84d9ee0e910ade78759fb OP_EQUAL
destination address: 2NANWHLLQDRA6vUMMbWyUNPxte8gWUMq719


In [4]:
printfn "%A" transaction 

{
  "hash": "516c43b617a6fa6dee5f3e9b34d99365b3b89d1381dc9aebcca1e8822c8cb5f4",
  "ver": 1,
  "vin_sz": 1,
  "vout_sz": 2,
  "lock_time": 0,
  "size": 250,
  "in": [
    {
      "prev_out": {
        "hash": "a8fd6a998d0d9bbf595b1a5d56d704d61cd42e0205e781286dcd723ebf41ed80",
        "n": 1
      },
      "scriptSig": "0014a0fd701a7079bba901b50cca88e2fd9e793ba255",
      "witness": "3045022100c768c68bb16feb17165075a2309ed152f9db4840b69fc3e1d9c69b504dde4b0202205fb1534b40ecaec0bb742a97004dab6ccfce45a22b80c5c4b08d46973804124801 0306ee7e04746dbe0a41b9258d62c6fde85dd7620b4be62a5e5ce768e5122aabb6"
    }
  ],
  "out": [
    {
      "value": "0.55000000",
      "scriptPubKey": "OP_DUP OP_HASH160 4fbd5152456bee3dc0ffe25118a6de17126e5c19 OP_EQUALVERIFY OP_CHECKSIG"
    },
    {
      "value": "30.30840894",
      "scriptPubKey": "OP_HASH160 bbdaa6cdc5793ab520c84d9ee0e910ade78759fb OP_EQUAL"
    }
  ]
}


[View transaction](http://tapi.qbit.ninja/transactions/516c43b617a6fa6dee5f3e9b34d99365b3b89d1381dc9aebcca1e8822c8cb5f4)

In [5]:
printfn "coinbase %A" transaction.IsCoinBase //find a coinbase transaction!

let mutable spentAmount:Money = Money.Zero

for spentCoin :TxOut  in outputsCoins do
    spentAmount <- spentAmount + spentCoin.Value

printfn "amount: %A" (spentAmount.ToDecimal(MoneyUnit.BTC))   

coinbase false
amount: 30.85840894M


In [6]:
let spentCoins:List<ICoin>  = transactionResponse.SpentCoins
let recCoins:List<ICoin>  = transactionResponse.ReceivedCoins

let mutable spentAmount:Money = Money.Zero

for spentCoin  in spentCoins do
    spentAmount <- (spentCoin.Amount.Add(spentAmount):?>Money)

printfn "amount: %A" (spentAmount.ToDecimal(MoneyUnit.BTC))   

amount: 30.85940894M


In [7]:
spentCoins.AddRange(recCoins)

let fee = transaction.GetFee(spentCoins.ToArray())

printfn "fee: %A" fee 


fee: 0.00100000


Get Balance

In [8]:
open System.Linq

printfn "%A" myBitcoinPubKeys.[0]
let balanceModel = client.GetBalance(myBitcoinPubKeys.[0], true).Result
if (balanceModel.Operations.Count = 0) then
    failwith "No coins to spend" 
    
 
let unspentCoins =
    query {
        for operation in  balanceModel.Operations do
            select operation.Amount
    }

let balance = unspentCoins |> Seq.sum 
 

printfn "balance: %A of %A" (balance.ToDecimal(MoneyUnit.BTC)) myBitcoinPubKeys.[0]


mnnaUEKBKJVFjCqVnVPmwefk6CwuVsyRRj
balance: 0.5298M of mnnaUEKBKJVFjCqVnVPmwefk6CwuVsyRRj
